In [8]:
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader
 
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
 
        token_ids = tokenizer.encode(txt)
 
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
 
    def __len__(self):
        return len(self.input_ids)
 
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [9]:
def create_dataloader_v1(txt, batch_size=4, 
        max_length=256, stride=128, shuffle=True, drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    return dataloader

In [10]:
with open("/workspaces/upper-division-cs/ai-24sp/assignments/whereismyprozac/data/totally-profound-monologue.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
 
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  22,   14, 1983,   14]]), tensor([[   14,  1983,    14, 37950]])]


In [11]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[   14,  1983,    14, 37950]]), tensor([[ 1983,    14, 37950,   198]])]


In [12]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4)
 
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[ 1042,    13,   198,   198],
        [19103,   510,    11,   475],
        [  338,  5340,    13,   314],
        [  422, 14745,     9,   198],
        [  986,   198,   198,  1026],
        [  290,   804,   379,   644],
        [ 9048, 10283,   523,  1593],
        [  550,  5284,    11,   290]])

Targets:
 tensor([[   13,   198,   198,  2061],
        [  510,    11,   475,   788],
        [ 5340,    13,   314,   892],
        [14745,     9,   198,   198],
        [  198,   198,  1026,   338],
        [  804,   379,   644,   356],
        [10283,   523,  1593,   284],
        [ 5284,    11,   290,   314]])


In [13]:
input_ids = torch.tensor([2, 3, 5, 1])

In [14]:
vocab_size = 6
output_dim = 3

In [15]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [16]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [17]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [18]:
output_dim = 256
vocab_size = 50257
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [19]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   22,    14,  1983,    14],
        [37950,   198,   198,  2215],
        [  314,   373,  1248,   986],
        [ 1248,   812,  1468,    11],
        [  314,  2497,   329,   262],
        [  717,   640,   287,   616],
        [ 1204,   986,   314,  2497],
        [  281,  2939,   286, 16287]])

Inputs shape:
 torch.Size([8, 4])


In [20]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [23]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [24]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
